In [ ]:
!pip install navec
!pip install pymystem3
!wget https://github.com/railbotan/pandas_task/raw/main/works.csv
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
# Mystem google collab fix
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

# ДЗ
Решение задачи с использованием NLP библиотек.
Основные: navec - для векторизации слов, Mystem - для лемматизации.
https://github.com/natasha/navec
https://yandex.ru/dev/mystem/
### Принцип работы:
#### Задание 1 (Сравнение специальности и места работы)
- Лемматизация входной строки (удаление знаков препинания, привод слов к начальной форме.)
- Векторизация слов.
- Поиск косинусного расстояния между словами.
- Вычисляем среднее всех косинусных расстояний.
#### Задание 2, 3
##### фильтрация датафрейма по принципу:
- Лемматизация входной строки.
- Поиск вхождений в лемматизированную строку.

In [17]:
import pandas as pd
import xml.etree.ElementTree as ET
from nltk.corpus import stopwords
from string import punctuation
from navec import Navec
import numpy as np
import scipy.spatial.distance as ds
from pymystem3 import Mystem
import nltk
nltk.download("stopwords")
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")
path = 'navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
--2021-12-12 04:46:46--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar.1’

navec_hudlit_v1_12B 100%[===================>]  50.56M  16.5MB/s    in 3.1s    

2021-12-12 04:46:50 (16.5 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar.1’ saved [53012480/53012480]



In [18]:
f = open("works.csv", encoding="utf-8")
cv = pd.read_csv(f).dropna()

In [19]:
m = Mystem()
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    return tokens

In [20]:
def vectorise_text(tokens):
    return [navec[t] for t in tokens if t in navec]

In [21]:
def compare_victorise_text(v_tokens1, v_tokens2):
    arr = []
    for i in v_tokens1:
        for j in v_tokens2:
            arr.append(ds.cosine(i, j))
    if len(arr) == 0:
        return(1)
    return np.mean(arr)

In [22]:
def fast_compare(str1, str2):
    tokens1 = preprocess_text(i)
    tokens2 = preprocess_text(j)
    v_tokens1 = vectorise_text(tokens1)
    v_tokens2 = vectorise_text(tokens2)
    return compare_victorise_text(v_tokens1, v_tokens2)

In [23]:
def filter_fn(row, search_str):
    return search_str in preprocess_text(row)

In [61]:
def get_top(count,df, return_value, search_str, filter_by):
    return df[df[filter_by].apply(lambda x: filter_fn(x, search_str))][return_value].str.lower().value_counts().head(count)


### У какого количества людей профессия и должность не совпадают
Пример смыслового сравнения строк на данном датасете. Где 0 - Максимальное совпадение, 1 - максимальная разность.

In [64]:
count = 0
for i, j in zip(cv[["jobTitle", "qualification"]].jobTitle, cv[["jobTitle", "qualification"]].qualification):
    comparation_res = fast_compare(i, j)
    if (comparation_res >= 0.72): count += 1
    print(i, "->", j, comparation_res, "-", count)
print("Итог:",count)

Медсестра -> Фельдшер 0.5186181366443634 - 0
Руководитель проектов -> Менеджер 0.6206537783145905 - 0
юрист -> юрист с отличием 0.4752016607671976 - 0
Ведущий специалист отдела сопровождения исполнения контрактов -> Магистр 0.9017206504164884 - 1
специалист по просроченной задолженности -> Художественный руководитель студии Декоративно-прикладного творчества, преподаватель 0.8946782189554402 - 2
Кассир-консультант -> Бакалавр 0.8008965440094471 - 3
Кредитный эксперт -> Экономист 0.7352558933198452 - 4
Начальник склада -> экономист по специальности "Бухгалтерский учет, анализ и аудит" 0.9287371191506585 - 5
Главный специалист отдела земельных отношений -> Бакалавр 0.9712797600775958 - 6
старший чабан -> нет 1 - 7
Ведущий Инженер-конструктор -> инженер 0.33541155854860943 - 7
помощник бурильщика КРС -> бакалавр 0.8409424610435963 - 8
Ведущий специалист службы электрических режимов -> Инженер 0.7358635663986206 - 9
педагог дополнительного образования -> Педагог - психолог 0.63955440372228

#### Итог: 718
P.S Всё зависит от порога которым мы выберем, в данном случае - 0.72
### Люди с каким образованием становятся менеджерами (топ-5):

In [62]:
print(get_top(5, cv, "qualification", "менеджер", "jobTitle"))

бакалавр              10
менеджер               8
экономист              6
специалист             6
экономист-менеджер     4
Name: qualification, dtype: int64


### Кем работают люди, которые по диплому являются инженерами (топ-5)

In [63]:
print(get_top(5, cv, "jobTitle", "инженер" , "qualification"))

главный инженер                3
заместитель директора          3
ведущий инженер-конструктор    2
инженер лесопользования        2
директор                       2
Name: jobTitle, dtype: int64
